In [3]:
path = '/Users/macbook/resize/train_label.csv'

file = open(path, 'w')

for i in  range(0,600):
    file.write(str(1)+'\n')
for i in  range(0,630):
    file.write(str(0)+'\n')
file.close()

In [5]:
import re
path1 = '/Users/macbook/jun'
file_list = os.listdir(path1)
print((file_list))

NameError: name 'os' is not defined

In [4]:
import os
import re
import cv2
import numpy as np
import csv
import random

#1. 데이터를 로드합니다.
train_image = '/Users/macbook/jun/'
train_label = '/Users/macbook/resize/train_label.csv'

#2. 필요한 모듈을 임폴트 한다. 
import loader9
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Activation
from sklearn.model_selection import train_test_split

#3. 하이퍼 파라미터를 설정한다. 
batch_size = 28
num_classes = 2 #설현과 수지라서 2
epochs = 100

#4. 데이터 로드 
x_train = loader9.image_load(train_image) #이미지를 numpy array형 숫자로 변환
y_train = loader9.label_load(train_label) #csv 파일을 읽어와서 one hot encoding

# 테스트 데이터
print ( loader9.image_load(train_image).shape )
print ( loader9.label_load(train_label).shape)

(1230, 128, 128, 3)
(1230, 2)


In [5]:
#5. 훈련데이터의 일부를 검증데이터로 구성합니다.
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, 
                                                  test_size = 0.3, 
                                                  random_state = 777)


# 모델 구성 
model = Sequential()

# convolution 1 floor
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation('relu'))

# convolution 2 floor
model.add(Conv2D(32, (3, 3))) #필터의 갯수: 32
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) #이미지 선명하게
model.add(Dropout(0.25))

# convolution 3 floor
model.add(Conv2D(64, (3, 3), padding='same')) #필터의 갯수: 64
model.add(BatchNormalization())
model.add(Activation('relu'))

# convolution 4 floor
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Fully connected floor
model.add(Flatten()) #완전연결계층에 입력할때는 flatten을 시켜준다
model.add(Dense(512)) #완전연결계층 1층의 뉴련의 개수: 512
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 출력층
model.add(Dense(num_classes)) #출력층의 뉴런의 개수: 2 (설현, 수지 분류)
model.add(Activation('softmax'))
 
# 모델 설정 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 
# 데이터 정규화 ----- 신경망과정에서 가장 중요함. 이상치에 민감하므로
x_train = x_train.astype('float32') #실수형으로 숫자를 전부 변환
#x_test = x_test.astype('float32')
x_train /= 255 #255으로 숫자를 나눠서 0~1사이의 숫자로 변환
#x_test /= 255
 
# 모델 학습 
hist = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs, batch_size=batch_size)
 
# 모델 평가 
scores = model.evaluate(x_train, y_train, verbose=0)

#[0.3352026641368866, 0.9929999709129333]
#    오차                        정확도 

print("CNN Error: %.2f%%" % (100-scores[1]*100))
 
save_model(model, "/Users/macbook/Desktop/test.h5") # 모델 저장 - 피클파일 생성과 같음

Epoch 1/100
31/31 [==============================] - 44s 1s/step - loss: 1.4515 - accuracy: 0.5467 - val_loss: 73.1160 - val_accuracy: 0.5312
Epoch 2/100
31/31 [==============================] - 42s 1s/step - loss: 0.7458 - accuracy: 0.6593 - val_loss: 46.1807 - val_accuracy: 0.5366
Epoch 3/100
31/31 [==============================] - 45s 1s/step - loss: 0.5525 - accuracy: 0.7284 - val_loss: 28.8125 - val_accuracy: 0.5312
Epoch 4/100
31/31 [==============================] - 43s 1s/step - loss: 0.3663 - accuracy: 0.8420 - val_loss: 18.6489 - val_accuracy: 0.5420
Epoch 5/100
31/31 [==============================] - 42s 1s/step - loss: 0.3346 - accuracy: 0.8609 - val_loss: 77.8831 - val_accuracy: 0.5339
Epoch 6/100
31/31 [==============================] - 42s 1s/step - loss: 0.2158 - accuracy: 0.9264 - val_loss: 43.6955 - val_accuracy: 0.4878
Epoch 7/100
31/31 [==============================] - 42s 1s/step - loss: 0.1289 - accuracy: 0.9695 - val_loss: 43.4218 - val_accuracy: 0.5528
Epoch 

Epoch 58/100
31/31 [==============================] - 44s 1s/step - loss: 0.0360 - accuracy: 0.9871 - val_loss: 681.5120 - val_accuracy: 0.4661
Epoch 59/100
31/31 [==============================] - 42s 1s/step - loss: 0.0342 - accuracy: 0.9935 - val_loss: 347.9689 - val_accuracy: 0.4986
Epoch 60/100
31/31 [==============================] - 49s 2s/step - loss: 0.0292 - accuracy: 0.9935 - val_loss: 767.3813 - val_accuracy: 0.4688
Epoch 61/100
31/31 [==============================] - 43s 1s/step - loss: 0.0194 - accuracy: 0.9947 - val_loss: 925.1005 - val_accuracy: 0.4688
Epoch 62/100
31/31 [==============================] - 50s 2s/step - loss: 0.0251 - accuracy: 0.9944 - val_loss: 633.4649 - val_accuracy: 0.4770
Epoch 63/100
31/31 [==============================] - 45s 1s/step - loss: 0.0282 - accuracy: 0.9938 - val_loss: 438.4108 - val_accuracy: 0.4743
Epoch 64/100
31/31 [==============================] - 45s 1s/step - loss: 0.0162 - accuracy: 0.9947 - val_loss: 595.3033 - val_accuracy: